# Bayesian Personalized Ranking
* See the corresponding file in `../../TrainingAlphas` for more details

In [ ]:
const source_name = "BPRT";

In [ ]:
using LightGBM
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");

In [ ]:
const ptw_items = Set(get_recommendee_split("ptw").item);

In [ ]:
function pairwise_evaluate(m, X, i, j)
    # returns >0 if i is preferred to j
    x = reshape(vcat(X[:, i], X[:, j]), 1, size(X)[1] * 2)
    l = predict(m, x)[1]
    y = reshape(vcat(X[:, j], X[:, i]), 1, size(X)[1] * 2)
    r = predict(m, y)[1]
    l - r
end;

In [ ]:
function get_model(source)
    params = read_params(source)
    X = []
    for alpha in params["alphas"]
        implicit = occursin("Implicit", alpha) && !occursin("Error", alpha)
        push!(X, read_recommendee_alpha(alpha, "all").rating)
    end
    X = collect(reduce(hcat, X)')
    m = params["model"]
    m, X
end;

In [ ]:
function compute_alpha()
    # unpack parameters
    df = read_recommendee_alpha("BPR", "all")
    ranking = df.item[sortperm(df.rating)]

    m_implicit, X_implicit = get_model("BPR.tree")
    m_ptw, X_ptw = get_model("BPR.tree.ptw")

    # fine tune the top of the rankings
    K = 1000
    safe = Set()
    @showprogress for _ = 1:K
        swaps = 0
        for i = 1:K-1
            if i in safe
                continue
            end
            if ranking[i] in ptw_items || ranking[i+1] in ptw_items
                comp = pairwise_evaluate(m_implicit, X_implicit, ranking[i], ranking[i+1])
            else
                comp = pairwise_evaluate(m_ptw, X_ptw, ranking[i], ranking[i+1])
            end
            if comp < 0
                ranking[i], ranking[i+1] = ranking[i+1], ranking[i]
                swaps += 1
                delete!(safe, i)
                delete!(safe, max(i - 1, 1))
            else
                push!(safe, i)
            end
        end
        if swaps == 0
            break
        end
    end

    # save alpha
    alpha = zeros(Float32, num_items())
    n = 1
    for i = 1:num_items()
        alpha[ranking[i]] = n
        n += 1
    end
    write_recommendee_alpha(alpha, source_name)
end;

In [ ]:
compute_alpha();